## Reddit Scraping
Documentation reference: https://praw.readthedocs.io/en/stable/

In [75]:
import requests
import pandas as pd
import praw
from praw.models import MoreComments

import os
import zipfile
import ast

os.makedirs("../output", exist_ok=True)
from dotenv import load_dotenv
load_dotenv()


True

# Scrape post from related subreddits
Usage: Consumer sentiment analysis 

In [74]:

reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
)

# "Skincare_Addiction", "asianskincare", "Blackskincare","SkincareAddicts"
subreddit_list = ["SkincareAddiction",]

all_posts = []

# get the top 20 post from each subreddit (don't know rate limit so 20 for now)
# Documentation: https://praw.readthedocs.io/en/stable/code_overview/models/subreddit.html
us_brands = ['Estée Lauder',
'Fenty Beauty (by Rihanna)',
'e.l.f. Cosmetics',
'Tarte Cosmetics',
'Glossier']

def get_top_comments(post):

    post.comments.replace_more(limit=0)  

    top_comments = []
    for comment in post.comments:
        if isinstance(comment, MoreComments):
            continue 
        top_comments.append(comment.body)
        if len(top_comments) == 5:
            break
    return top_comments
        
for sub in subreddit_list:
    try:
        for brand in us_brands:
            query = f'"{brand}"'
            post_collection = reddit.subreddit(sub).search(query, limit=5)  
            for post in post_collection: 
                top_comments = get_top_comments(post)
                all_posts.append({
                "subreddit_name": sub,
                "post_id": post.id,
                "title": post.title,
                "description": post.selftext,
                "score": post.score,
                "num_comments": post.num_comments,
                "top_comments": top_comments,
                "upvote_ratio": post.upvote_ratio,
                "brand": brand
            })
    except Exception as e:
        print(f"Error scraping {sub}: {e}")
        continue
    
subreddit_df = pd.DataFrame(all_posts)
subreddit_df
subreddit_df.to_csv("../output/subreddit_data.csv", index=False)



In [84]:
skincare_df = pd.read_csv("../output/subreddit_data.csv")
skincare_df["top_comments_cleaned"] = skincare_df["top_comments"].apply(ast.literal_eval)
skincare_df["top_comments_cleaned"][0] # make it a bit more readable 

skincare_df

,subreddit_name,post_id,title,description,score,num_comments,top_comments,upvote_ratio,brand,top_comments_cleaned
0,SkincareAddiction,8qkw0m,[B&A] FIRST TIME POSTER!!! Only 2 months of us...,NaN,5076,268,['Your skin looks great but the best part of t...,0.98,Estée Lauder,[Your skin looks great but the best part of th...
1,SkincareAddiction,8fci2y,[MISC] Brandon Truaxe sent a series of emails ...,NaN,182,152,"['""No need to discuss"" um WAT? Yes, yes need. ...",0.93,Estée Lauder,"[""No need to discuss"" um WAT? Yes, yes need. W..."
2,SkincareAddiction,1i5q8th,[Product question] Dupe for Estée Lauder Advan...,I got this half off at Ulta based on the esthe...,8,7,['\nAnnouncing: AMA with The Ordinary on Jan 1...,0.91,Estée Lauder,[\nAnnouncing: AMA with The Ordinary on Jan 16...
3,SkincareAddiction,ttbud1,[Misc] Anyone got any tips/product recommendat...,NaN,35,54,['Hi everyone and welcome to SkincareAddiction...,0.91,Estée Lauder,[Hi everyone and welcome to SkincareAddiction!...
4,SkincareAddiction,xyfyv9,[Product Question] Is there an eye repair crea...,I’m required to wear makeup five days a week f...,9,45,['Hi everyone and welcome to SkincareAddiction...,0.84,Estée Lauder,[Hi everyone and welcome to SkincareAddiction!...
5,SkincareAddiction,tf1ido,[Review] e.l.f. Cosmetics Holy Hydration! Make...,**Product name**: e.l.f. Cosmetics Holy Hydra...,27,18,['Hi everyone and welcome to SkincareAddiction...,0.92,e.l.f. Cosmetics,[Hi everyone and welcome to SkincareAddiction!...
6,SkincareAddiction,1h0n7w3,[Misc] I drew my skincare routine! Reviews in ...,NaN,721,43,['Are you brand new to skincare? The best plac...,0.97,e.l.f. Cosmetics,[Are you brand new to skincare? The best place...
7,SkincareAddiction,pmg5as,[Product Question] Thoughts on e.l.f. Cosmetic...,Thoughts about this product? Needing a new exf...,6,2,['Hi everyone and welcome to SkincareAddiction...,1.00,e.l.f. Cosmetics,[Hi everyone and welcome to SkincareAddiction!...
8,SkincareAddiction,khzaxg,[product question] e.l.f. Cosmetics Superhydra...,I have never tried this product before. I'm ho...,0,1,['[Nominations for **Best of SkincareAddiction...,0.50,e.l.f. Cosmetics,[[Nominations for **Best of SkincareAddiction ...
9,SkincareAddiction,9byd25,[PSA] E.L.F. Cosmetics/skincare sale,NaN,12,2,"[""What are your favorite products from ELF? I ...",0.88,e.l.f. Cosmetics,[What are your favorite products from ELF? I r...
